In [1]:
import os

os.environ["OPENAI_API_KEY"] = '[OPENAI_API_KEY]'

In [2]:
# !conda install unstructured
from langchain.document_loaders import TextLoader
markdown_path = "./notion_bookmarks/bookmarks.md"
loader = TextLoader(markdown_path, encoding='utf-8')
document = loader.load()

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
texts = text_splitter.split_documents(document)
# texts
# dir(texts[0].page_content)
# texts[0].page_content

In [4]:
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

# embeddings

In [5]:
from langchain.vectorstores import Chroma

db = Chroma.from_documents(texts, embeddings, persist_directory="./db/")
# db

# query = "property and crypto currency"
# docs = db.similarity_search(query, k=4)
# docs

Using embedded DuckDB with persistence: data will be stored in: ./db/


In [6]:
retriever = db.as_retriever()
# retriever.search_kwargs['distance_metric'] = 'cos'
# retriever.search_kwargs['fetch_k'] = 20
# retriever.search_kwargs['maximal_marginal_relevance'] = True
# retriever.search_kwargs['k'] = 20

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

model = ChatOpenAI(model_name='gpt-3.5-turbo') # 'ada' 'gpt-3.5-turbo' 'gpt-4',
qa = ConversationalRetrievalChain.from_llm(model, retriever=retriever)

In [10]:
questions = [
    "Can you categorize the content into at least 6 categories?",
    "Of all the contenct, how many distinct urls are there?",
    # "Show me the first 20 lines",
    # "For those categories, may you provide examples of items that would fall under each category?",
    # "What one improvement do you propose in code in relation to the class herarchy for the Chain class?",
] 
chat_history = []

for question in questions:  
    result = qa({"question": question, "chat_history": chat_history})
    chat_history.append((question, result['answer']))
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n")

-> **Question**: Can you categorize the content into at least 6 categories? 

**Answer**: Based on the given context, I can categorize the content into two categories:
1. Articles: It includes links to two articles - "Distributed Systems Reading List" and "Catalog of Patterns of Enterprise Application Architecture".
2. Blog: It includes a link to "Second Enumerations" blog. 

-> **Question**: Of all the contenct, how many distinct urls are there? 

**Answer**: I'm sorry, but I cannot determine the number of distinct URLs included in the content without additional information. The pieces of context provided do not give a clear idea about the content or specific URLs. 



Later

from langchain import OpenAI
from langchain.chains.summarize import load_summarize_chain

llm = OpenAI(temperature=0)
summary_chain = load_summarize_chain(llm, chain_type="map_reduce")